In [1]:
import os
import io
import json
import librosa
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
#Loads training dataset from json file
with open("data.json", "r") as fp:
    data = json.load(fp)

X = np.array(data["MFCCs"])
y = np.array(data["labels"])

In [3]:
# create train, validation, test split
train_img, test_img, train_label, test_label = train_test_split(X, y, test_size=0.2)
train_img, validation_img, train_label, validation_label = train_test_split(train_img, train_label, test_size=0.2)

In [4]:
# add an axis to nd array
train_img = train_img[..., np.newaxis]
test_img = test_img[..., np.newaxis]
validation_img = validation_img[..., np.newaxis]

In [5]:
input_shape = (train_img.shape[1], train_img.shape[2], 1)

In [6]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation="relu", input_shape=input_shape))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1,1), padding='same'))
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(128, kernel_size=(3,3), activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Flatten()) # Flattening the 2D arrays for fully connected layers
model.add(tf.keras.layers.Dense(128, activation="relu"))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(64, activation="relu"))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(32, activation="relu"))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(10,activation="softmax"))
# print model parameters on console
#model.summary()

In [7]:
# compile model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

#train the model
model.fit(train_img, train_label, epochs=50 , validation_data=(validation_img, validation_label))

Epoch 1/50
427/427 [==============================] - 34s 80ms/step - loss: 2.0691 - accuracy: 0.2447 - val_loss: 1.3663 - val_accuracy: 0.5199
Epoch 2/50
427/427 [==============================] - 34s 80ms/step - loss: 1.2805 - accuracy: 0.5173 - val_loss: 0.8622 - val_accuracy: 0.7006
Epoch 3/50
427/427 [==============================] - 36s 85ms/step - loss: 0.8871 - accuracy: 0.6814 - val_loss: 0.5798 - val_accuracy: 0.8026
Epoch 4/50
427/427 [==============================] - 37s 86ms/step - loss: 0.6464 - accuracy: 0.7774 - val_loss: 0.4218 - val_accuracy: 0.8630
Epoch 5/50
427/427 [==============================] - 37s 86ms/step - loss: 0.5325 - accuracy: 0.8286 - val_loss: 0.4184 - val_accuracy: 0.8669
Epoch 6/50
427/427 [==============================] - 37s 86ms/step - loss: 0.4407 - accuracy: 0.8589 - val_loss: 0.3941 - val_accuracy: 0.8727
Epoch 7/50
427/427 [==============================] - 37s 86ms/step - loss: 0.3822 - accuracy: 0.8790 - val_loss: 0.3496 - val_accuracy:

In [8]:
# evaluate network on test set
test_loss, test_acc = model.evaluate(test_img, test_label)
print("loss: " , test_loss)
print("accuracy: ", test_acc)

134/134 [==============================] - 3s 20ms/step - loss: 0.5868 - accuracy: 0.9125
loss:  0.5868441462516785
accuracy:  0.9125029444694519


In [9]:
#make prediction
file_path = "up.wav"
signal, sample_rate = librosa.load(file_path)


In [10]:
SAMPLES_TO_CONSIDER = 22050 #samples in 1 sec
if len(signal) >= SAMPLES_TO_CONSIDER:
            # ensure consistency of the length of the signal
            signal = signal[:SAMPLES_TO_CONSIDER]

            # extract MFCCs
            MFCCs = librosa.feature.mfcc(signal, sample_rate)
            
MGCCs = MFCCs.T

# we need a 4-dim array to feed to the model for prediction: (# samples, # time steps, # coefficients, 1)
MFCCs = MFCCs[np.newaxis, ..., np.newaxis]

In [11]:
mapping = np.array(data["mapping"])
print(mapping)

['down' 'go' 'left' 'no' 'off' 'on' 'right' 'stop' 'up' 'yes']


In [12]:
# get the predicted label
predictions = model.predict(MFCCs)[0]
print(predictions)
predicted_index = np.argmax(predictions)
predicted_keyword = mapping[predicted_index]
print(predicted_keyword)

[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
off


In [13]:
res = {mapping[i]: round(predictions[i]*100,2) for i in range(len(mapping))}
print(str(res))

{'down': 0.0, 'go': 0.0, 'left': 0.0, 'no': 0.0, 'off': 100.0, 'on': 0.0, 'right': 0.0, 'stop': 0.0, 'up': 0.0, 'yes': 0.0}


In [14]:
file_path = "speech_test/stop.wav"
signal, sample_rate = librosa.load(file_path)


In [15]:
SAMPLES_TO_CONSIDER = 22050 #samples in 1 sec
if len(signal) >= SAMPLES_TO_CONSIDER:
            # ensure consistency of the length of the signal
            signal = signal[:SAMPLES_TO_CONSIDER]
           # extract MFCCs
MFCCs = librosa.feature.mfcc(signal, sample_rate)
            
MGCCs = MFCCs.T

# we need a 4-dim array to feed to the model for prediction: (# samples, # time steps, # coefficients, 1)
MFCCs = MFCCs[np.newaxis, ..., np.newaxis]

In [16]:
# get the predicted label
predictions = model.predict(MFCCs)[0]
print(predictions)
predicted_index = np.argmax(predictions)
predicted_keyword = mapping[predicted_index]
print(predicted_keyword)

[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
off


In [17]:
res = {mapping[i]: round(predictions[i]*100,2) for i in range(len(mapping))}
print(str(res))

{'down': 0.0, 'go': 0.0, 'left': 0.0, 'no': 0.0, 'off': 100.0, 'on': 0.0, 'right': 0.0, 'stop': 0.0, 'up': 0.0, 'yes': 0.0}
